# Notebook 4: Projecting Injections

In this notebook we will explore how we can use GravyFlow GPU functions to project injections onto the GPU with physically in a physically realistic fashion. As ususal we wills start by performing the neccisary imports:

In [1]:
# Built-in imports
from typing import List
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

2024-08-20 09:52:33.678689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Then setting up the GPU environment:

In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env()

INFO:root:TensorFlow version: 2.12.1, CUDA version: 11.8
2024-08-20 09:52:49.211638: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-20 09:52:49.211701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In order to demonstrate projection onto a detector network, we will need example waveforms. We can use the cuPhenomGenerator we learned about in notebook 3 to generate example waveforms:

In [3]:
# Create a waveform generator to generate example IMRPhenomD waveforms:
phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)
phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:

    # Generate one exammple waveform with the generator:
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

2024-08-20 09:52:50.625610: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x5585bca95430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-20 09:52:50.625651: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-08-20 09:52:50.653929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-08-20 09:52:50.674039: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


## Projecting Injections onto Multiple Detectors

GravyFlow uses the `gf.Network` class to handle networks of virtual gravitational wave detectors.

`gf.Network` has the following initilisation parameters:

- `parameters` : `Union[List[IFO], Dict]`, Required
  > There are two ways to define the detector network when intilising a `gf.Network` object. A list of interferometers already hard coded in to GravyFlow, or a dictionary containing custom interferometers.

- `seed` : `int` = `None`
  > The seed for the random number generators used when projecting injections. If set to None, the seed from `gf.Defaults` will be used.

First, we will initlise a detector object with a list of detectors already hard coded into GravyFlow:

In [4]:
# Generate a gf.Network object that constists of the Livinston (L1), Hanford (H1), and Virgo (V1) detectors.
network : gf.Network = gf.Network(
    parameters=[gf.IFO.L1, gf.IFO.H1, gf.IFO.V1]
)

2024-08-20 09:52:51.011509: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


With this `gf.Network` object, we can project the injections that we previously generated, using the `project_wave` class function, which has the following arguments:

- `strain` : `tf.Tensor`, Required
  > A `tf.Tensor` containing batches of both polarizations of raw strain of the injection that you wish to project.

- `sample_rate_hertz` : `Optional[float]` = `None`, Optional
  > The sample rate in Hertz of the input strain. Defaults to the value sample rate set in `gf.Defaults`.

- `right_ascension` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The right ascesnion of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

- `declination` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The declination of the simualted source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

-  `polarization` : `Optional[Union[tf.Tensor, List[float], float]]` = `None`, Optional
  > The polarizaton of the simulated source of the gravitational wave, which will be used for the projection of the injection. By default, the sources will be randomly distributed across the sky.

Let's project the world using a toy value of 0.0 for all of the three sky location and polarization parameters: `right_ascension`, `declination`, `polarization`.

In [6]:
# Use the TensorFlow environment generated earlier:
with env:

    # Project the waveform contained in phenom_d_injection onto the detectors 
    # defined in network.projection_wave:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0],
        right_ascension=0.0,
        declination=0.0,
        polarization=0.0
    )

2024-08-20 09:53:16.483456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_4815}}{{node PartitionedCall}}]]
2024-08-20 09:53:17.638350: I tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:328] ptxas warning : Registers are spilled to local memory in function 'fusion_3', 8 bytes spill stores, 8 bytes spill loads



We can then plot the output of the waveform projection, the projection will create one channel for each detector, and will no longer contain both polarizations as they have been reduced in the projection.

In [7]:
# Since we are only projecting one injection, extract the first injection:
projected_injection : tf.Tensor = projected_injections[0]

# Plot the three channel output of the projection.
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Projecting from a random Sky Location and Polarization:

Rather than inputing a specific sky localisation and polarization and direction. The default behaviour of `project_wave` is to project the waveform from a random sky direction and polarization.

We can see that below:

In [ ]:
# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-06-19 11:31:43.188823: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_injections' with dtype int32
	 [[{{node num_injections}}]]
2024-06-19 11:31:43.198147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_injections' with dtype int32
	 [[{{node num_injections}}]]
2024-06-19 11:31:43.250238: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_4815}}{{node PartitionedCall}}]]


(3, 4096)


Loading BokehJS ...

## Adding custom detectors

We can also add custom detectors using a dictionary, which we can see in the example below:

In [8]:
# Adding Custom Detectors:

# Create from dictionary:
network = gf.Network({
    "longitude_radians" : [np.pi/2, np.pi/4], 
    "latitude_radians" : [-np.pi/4, np.pi/6],
    "y_angle_radians" : [(2*np.pi/3), np.pi], 
    "x_angle_radians" : None, 
    "height_meters" : [0.0, 0.0],
    "x_length_meters" : [4000.0, 10000.0],
    "y_length_meters" : [4000.0, 10000.0]
})

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-08-20 09:53:23.812228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_injections' with dtype int32
	 [[{{node num_injections}}]]
2024-08-20 09:53:23.817760: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_injections' with dtype int32
	 [[{{node num_injections}}]]
2024-08-20 09:53:23.935124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_shift_waveform_5251}}{{node PartitionedCall}}]]


Loading BokehJS ...

## Load Detector From Config

We can also load a custom network from a `.json` file, as in the example below:

In [9]:
# Define injection directory path:
example_network_directory : Path =  Path("./example_configs/example_network.json")

network = gf.Network.load(example_network_directory)

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Incoherent Injections

We can generate incoherent injections by using a `gf.IncoherentGenerator` composed of other waveform generators, which allows us to use a different waveform in each detector. Note that if we do this we must be carefull to use the same number of component waveforms as there are detectors in the network we use to project with.

In [10]:
wnb_generator_a : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50.0,
    max_frequency_hertz=100.0
)
phenom_d_generator_a : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=10.0
)
phenom_d_generator_b : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=10.0,
    mass_2_msun=10.0,
    inclination_radians=20.0
)

incoherent_generator : gf.InjectionGenerator  = gf.IncoherentGenerator(
    [wnb_generator_a, phenom_d_generator_a, phenom_d_generator_b]
)

incoherent_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(incoherent_generator)    

Finally, we can plot these incoherent injections:

In [11]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.H1])
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    incoherent_injections, _, _ = next(
        incoherent_injection_generator(num_examples_per_batch=1)
    )        

    projected_injections : tf.Tensor = network.project_wave(
        incoherent_injections
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-08-20 09:53:34.098815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_waveforms' with dtype int32
	 [[{{node num_waveforms}}]]
2024-08-20 09:53:34.116600: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'hann_window/cond/ones/packed/Cast_1' with dtype int32
	 [[{{node hann_window/cond/ones/packed/Cast_1}}]]
2024-08-20 09:53:34.219800: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'foldl/while/hann_window/cond/ones/packed/fold

Loading BokehJS ...